# THE BATTLE OF NEIGHBORHOODS PROJECT

This notebook has the code for the final project.

In [1]:
import numpy as np
import pandas as pd

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Find the city with the largest Chinese community

In [2]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_60fde9a69b664de08697ac05914adba4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6qB0inNHMBEdQXk12ENH3P9gPJ2LQSvtZj_ilCDdIrg4',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_60fde9a69b664de08697ac05914adba4.get_object(Bucket='capstone-donotdelete-pr-rtwgqykbmttzjy',Key='cenVisibleMinorityDA2016.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# load census data
census2016 = pd.read_csv(body, header = 0)
census2016.head()

,OBJECTID,DAUID,CSDUID,CSDNAME,TOT_VISIBLE_MINORITY_PVT_HH,TOT_VISIBLE_MINORITY_POP,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,...,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE,VISIBLE_MINORITY_NIE,MULTIPLE_VISIBLE_MINORITY,NOT_VISIBLE_MINORITY,SHAPE.STArea(),SHAPE.STLength()
0,1,35191319,3519028,Vaughan,1055.0,440.0,55.0,130.0,15.0,70.0,...,0.0,10.0,25.0,125.0,0.0,10.0,0.0,615.0,1.657006e+05,1881.204066
1,2,35191314,3519028,Vaughan,1230.0,490.0,95.0,210.0,0.0,35.0,...,0.0,0.0,45.0,45.0,0.0,15.0,40.0,740.0,6.353679e+05,3568.832137
2,3,35191037,3519028,Vaughan,6330.0,2565.0,925.0,505.0,225.0,335.0,...,120.0,85.0,135.0,45.0,20.0,45.0,25.0,3765.0,1.405759e+06,5055.080921
3,4,35190824,3519028,Vaughan,2150.0,455.0,280.0,70.0,35.0,15.0,...,0.0,20.0,10.0,0.0,0.0,25.0,10.0,1700.0,6.461407e+05,4272.214232
4,5,35190825,3519028,Vaughan,565.0,100.0,70.0,0.0,0.0,0.0,...,0.0,10.0,10.0,0.0,0.0,0.0,0.0,465.0,2.537939e+05,2756.348141


### clean the census data

In [3]:
# drop the irrelevant columns
column_names = census2016.columns.values
drop_index = [0, 1, 2, 4, 5, 16, 17, 18, 19, 20]
drop_columns = []
for idx in drop_index:
    drop_columns.append(column_names[idx])
census2016.drop(labels = drop_columns, axis = 1, inplace = True)
census2016.head()

,CSDNAME,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
0,Vaughan,55.0,130.0,15.0,70.0,0.0,0.0,10.0,25.0,125.0,0.0
1,Vaughan,95.0,210.0,0.0,35.0,0.0,0.0,0.0,45.0,45.0,0.0
2,Vaughan,925.0,505.0,225.0,335.0,105.0,120.0,85.0,135.0,45.0,20.0
3,Vaughan,280.0,70.0,35.0,15.0,0.0,0.0,20.0,10.0,0.0,0.0
4,Vaughan,70.0,0.0,0.0,0.0,15.0,0.0,10.0,10.0,0.0,0.0


In [4]:
# change CSDNAME to City
census2016.rename(columns = {'CSDNAME': 'CITY'}, inplace = True)
census2016.head()

,CITY,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
0,Vaughan,55.0,130.0,15.0,70.0,0.0,0.0,10.0,25.0,125.0,0.0
1,Vaughan,95.0,210.0,0.0,35.0,0.0,0.0,0.0,45.0,45.0,0.0
2,Vaughan,925.0,505.0,225.0,335.0,105.0,120.0,85.0,135.0,45.0,20.0
3,Vaughan,280.0,70.0,35.0,15.0,0.0,0.0,20.0,10.0,0.0,0.0
4,Vaughan,70.0,0.0,0.0,0.0,15.0,0.0,10.0,10.0,0.0,0.0


In [5]:
# group the data by city
census2016_sum = census2016.groupby(by = 'CITY').sum()
census2016_sum

,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
CITY,,,,,,,,,,
Aurora,1840.0,5575.0,1200.0,935.0,670.0,575.0,300.0,1870.0,595.0,195.0
Chippewas of Georgina Island First Nation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
East Gwillimbury,470.0,690.0,335.0,165.0,100.0,85.0,70.0,255.0,90.0,60.0
Georgina,685.0,670.0,695.0,335.0,375.0,100.0,130.0,185.0,95.0,70.0
King,930.0,445.0,395.0,160.0,170.0,125.0,140.0,320.0,160.0,70.0
Markham,58255.0,147710.0,9700.0,8945.0,1730.0,3240.0,2460.0,7940.0,4350.0,1015.0
Newmarket,3265.0,5565.0,2295.0,1835.0,1225.0,660.0,2275.0,1970.0,695.0,325.0
Richmond Hill,14910.0,56970.0,3850.0,3790.0,1580.0,3565.0,1415.0,20120.0,5485.0,625.0
Vaughan,30655.0,20790.0,8345.0,8645.0,7415.0,4325.0,6840.0,8650.0,5360.0,300.0


In [6]:
# sort by number of Chinese
census2016_sum.sort_values(by = ['CHINESE'], axis = 0, ascending = False, inplace = True)
census2016_sum

,SOUTH_ASIAN,CHINESE,BLACK,FILIPINO,LATIN_AMERICAN,ARAB,SOUTHEAST_ASIAN,WEST_ASIAN,KOREAN,JAPANESE
CITY,,,,,,,,,,
Markham,58255.0,147710.0,9700.0,8945.0,1730.0,3240.0,2460.0,7940.0,4350.0,1015.0
Richmond Hill,14910.0,56970.0,3850.0,3790.0,1580.0,3565.0,1415.0,20120.0,5485.0,625.0
Vaughan,30655.0,20790.0,8345.0,8645.0,7415.0,4325.0,6840.0,8650.0,5360.0,300.0
Whitchurch-Stouffville,5625.0,5965.0,995.0,1135.0,330.0,435.0,295.0,335.0,190.0,150.0
Aurora,1840.0,5575.0,1200.0,935.0,670.0,575.0,300.0,1870.0,595.0,195.0
Newmarket,3265.0,5565.0,2295.0,1835.0,1225.0,660.0,2275.0,1970.0,695.0,325.0
East Gwillimbury,470.0,690.0,335.0,165.0,100.0,85.0,70.0,255.0,90.0,60.0
Georgina,685.0,670.0,695.0,335.0,375.0,100.0,130.0,185.0,95.0,70.0
King,930.0,445.0,395.0,160.0,170.0,125.0,140.0,320.0,160.0,70.0


We now know that Markham has the largest Chinese community in York Region. Let us sort the number in Markham to see what the numbers are for each visible minority group.

In [7]:
# sort the columns by the values in Markham
census2016_sum.sort_values(by = ['Markham'], axis = 1, ascending = False, inplace = True)
census2016_sum

,CHINESE,SOUTH_ASIAN,BLACK,FILIPINO,WEST_ASIAN,KOREAN,ARAB,SOUTHEAST_ASIAN,LATIN_AMERICAN,JAPANESE
CITY,,,,,,,,,,
Markham,147710.0,58255.0,9700.0,8945.0,7940.0,4350.0,3240.0,2460.0,1730.0,1015.0
Richmond Hill,56970.0,14910.0,3850.0,3790.0,20120.0,5485.0,3565.0,1415.0,1580.0,625.0
Vaughan,20790.0,30655.0,8345.0,8645.0,8650.0,5360.0,4325.0,6840.0,7415.0,300.0
Whitchurch-Stouffville,5965.0,5625.0,995.0,1135.0,335.0,190.0,435.0,295.0,330.0,150.0
Aurora,5575.0,1840.0,1200.0,935.0,1870.0,595.0,575.0,300.0,670.0,195.0
Newmarket,5565.0,3265.0,2295.0,1835.0,1970.0,695.0,660.0,2275.0,1225.0,325.0
East Gwillimbury,690.0,470.0,335.0,165.0,255.0,90.0,85.0,70.0,100.0,60.0
Georgina,670.0,685.0,695.0,335.0,185.0,95.0,100.0,130.0,375.0,70.0
King,445.0,930.0,395.0,160.0,320.0,160.0,125.0,140.0,170.0,70.0


We now know that Markham has the largest Chinese community. The numbers for other communities are useful when we select the food that we will serve. We should now look at the neighborhoods in Markham.

## 2. Explore the neighborhoods in Markham

In [16]:
# load the postal codes and neighborhoods data
body2 = client_60fde9a69b664de08697ac05914adba4.get_object(Bucket='capstone-donotdelete-pr-rtwgqykbmttzjy',Key='postalcode_neighborhood.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

markham_neigh = pd.read_excel(body2)
markham_neigh

,CITY,POSTALCODE,NEIGHBORHOOD,LATITUDE,LONGITUDE
0,MARKHAM,L3P,Markham Central,43.8793,-79.2620
1,MARKHAM,L3R,Markham Outer Southwest,43.8552,-79.3071
2,MARKHAM,L3S,Markham Southeast,43.8509,-79.2620
3,MARKHAM,L6B,Markham East,43.8687,-79.2254
4,MARKHAM,L6C,Markham Northwest,43.8783,-79.3522
5,MARKHAM,L6E,Markham Northeast,43.8992,-79.2733
6,MARKHAM,L6G,Markham Inner Southwest,43.8458,-79.3325


In [17]:
# see Markham neighborhoods on a map
address = 'Markham, ON'
geolocator = Nominatim(user_agent = "on_explorer")
location = geolocator.geocode(address)
latitude_markham = location.latitude
longitude_markham = location.longitude

# create map object
map_markham = folium.Map(location=[latitude_markham, longitude_markham], zoom_start=11)

# add markers to map
for lat, lng, label in zip(markham_neigh['LATITUDE'], markham_neigh['LONGITUDE'], markham_neigh['NEIGHBORHOOD']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_markham)  
    
map_markham

### find the venues around the neighborhoods from Foursquare

In [18]:
# @hidden_cell
CLIENT_ID = 'A01AQLLXBB0MJQWQLC24ZAWIFXWXCFINCDY1Q5PDVSEI25GO' # your Foursquare ID
CLIENT_SECRET = 'R5GWUOYPHOYFXRRZ2235RRLXA1RVCWMXLZXRXPQEBCD14IXE' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

In [27]:
#'''
neighborhood_latitude = markham_neigh.loc[4, 'LATITUDE'] # neighborhood latitude value
neighborhood_longitude = markham_neigh.loc[4, 'LONGITUDE'] # neighborhood longitude value

neighborhood_name = markham_neigh.loc[4, 'NEIGHBORHOOD'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# type your answer here
LIMIT = 150
radius = 3500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()#['response']['groups'][0]['items']
results
#'''

Latitude and longitude values of Markham Northwest are 43.8783, -79.3522.


{'meta': {'code': 200, 'requestId': '5ea6226e94939340cc898ddc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Markham',
  'headerFullLocation': 'Markham',
  'headerLocationGranularity': 'city',
  'totalResults': 162,
  'suggestedBounds': {'ne': {'lat': 43.90980003150003,
    'lng': -79.30858093676625},
   'sw': {'lat': 43.84679996849997, 'lng': -79.39581906323374}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53a60abb498ea7107daaff79',
       'name': 'Hub Climbing Markham',
       'location': {'address': '165 McIntosh Drive',
        'crossStreet': 'Apple Creek and Woodbine Road',
        'lat': 43.85712899823114,
        'lng': -79.35770257931846,
        'labeledLatLngs': 

In [41]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT = 100, radius = 1000):
    
    '''
    
    names: neighborhood names,
    latitudes, longitudes: geospatial coordinates of the neighborhoods,
    Limit: maximum number of venues returned,
    radius: search radius in meters.
    
    '''
    
    venues_list = []
    count = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])
        count += 1

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue ID']
    
    #show number of neighborhood explored
    print('\n'+ " ".join([str(count), 'neighborhoods explored.']))
    
    return(nearby_venues)

In [42]:
markham_venues = getNearbyVenues(names = markham_neigh['NEIGHBORHOOD'],
                                   latitudes = markham_neigh['LATITUDE'],
                                   longitudes = markham_neigh['LONGITUDE'],
                                  LIMIT = 150, radius = 3000
                                )
print(markham_venues.shape)
markham_venues.head()

Markham Central
Markham Outer Southwest
Markham Southeast
Markham East
Markham Northwest
Markham Northeast
Markham Inner Southwest

7 neighborhoods explored.
(607, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Markham Central,43.8793,-79.262,Inspire Restaurant,43.879232,-79.261249,Asian Restaurant,53fe575d498eb015fb81f5a7
1,Markham Central,43.8793,-79.262,Rouge River Brewing Company,43.878876,-79.265976,Brewery,536e1f8b498eaf1e396dd0a5
2,Markham Central,43.8793,-79.262,Love Gelato,43.876927,-79.260213,Ice Cream Shop,51c1b3168bbd62da8dc78db0
3,Markham Central,43.8793,-79.262,The Duchess of Markham,43.876159,-79.260219,Gastropub,4b155b19f964a5207aab23e3
4,Markham Central,43.8793,-79.262,Main St. Greek,43.876545,-79.260666,Greek Restaurant,573f87d8498e0114228d27b1


The radius for the search was quite large, so we need to drop duplicates.

In [44]:
markham_venues.drop_duplicates(subset = ['Venue ID'], keep = 'first', inplace = True)
print(markham_venues.shape)

(377, 8)


### explore the venues

In [47]:
# check numbers of venues returned for each neighborhood
markham_venues_grouped = markham_venues.groupby('Neighborhood')[['Venue']].count()
markham_venues_grouped.sort_values(by = 'Venue', axis = 0, ascending = False, inplace = True)
markham_venues_grouped

,Venue
Neighborhood,
Markham Central,100
Markham Outer Southwest,79
Markham Northwest,62
Markham Southeast,49
Markham Inner Southwest,44
Markham Northeast,32
Markham East,11


In [63]:
markham_central = markham_venues[markham_venues['Neighborhood'] == 'Markham Central'].reset_index(drop = True)
markham_outer_southwest = markham_venues[markham_venues['Neighborhood'] == 'Markham Outer Southwest'].reset_index(drop = True)
markham_northwest = markham_venues[markham_venues['Neighborhood'] == 'Markham Northwest'].reset_index(drop = True)
markham_southeast = markham_venues[markham_venues['Neighborhood'] == 'Markham Southeast'].reset_index(drop = True)
markham_inner_southwest = markham_venues[markham_venues['Neighborhood'] == 'Markham Inner Southwest'].reset_index(drop = True)
markham_northeast = markham_venues[markham_venues['Neighborhood'] == 'Markham Northeast'].reset_index(drop = True)
markham_east = markham_venues[markham_venues['Neighborhood'] == 'Markham East'].reset_index(drop = True)
markham_east

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Markham East,43.8687,-79.2254,Starbucks,43.866234,-79.229210,Coffee Shop,5cb02fa389e490002c5e57b9
1,Markham East,43.8687,-79.2254,Subway,43.867134,-79.229632,Sandwich Place,4b58ed08f964a520727328e3
2,Markham East,43.8687,-79.2254,Dollarama,43.867239,-79.230952,Discount Store,4ae3756af964a5205e9521e3
3,Markham East,43.8687,-79.2254,Walmart,43.871373,-79.214939,Big Box Store,4c93ca8582b56dcb0808c9aa
4,Markham East,43.8687,-79.2254,McDonald's,43.871729,-79.215787,Fast Food Restaurant,506b7151e4b0b56e416fc3bd
5,Markham East,43.8687,-79.2254,Eggs 4 Life,43.867153,-79.230545,Breakfast Spot,4d860e8161676dcb081797e4
6,Markham East,43.8687,-79.2254,Tasty Shawarma,43.853233,-79.254320,Middle Eastern Restaurant,5256ec1b498eb9dc27fee038
7,Markham East,43.8687,-79.2254,La Jawab Grill,43.850953,-79.253531,Indian Restaurant,513bd9b2e4b0e8ef82cee9c9
8,Markham East,43.8687,-79.2254,Rouge Valley Park (Markham),43.870302,-79.250350,Trail,4c479597699476b0a14db221
9,Markham East,43.8687,-79.2254,Svengali Hair Salon,43.875239,-79.259818,Cosmetics Shop,4c791b512d3ba143eff38cd0
